# अंश 2: फेडरेटेड लर्निंग (Federated Learning) में परिचय

अंतिम खंड में, हमने Pointer Tensor के बारे में सीखा, जो कि डीप लर्निंग (Deep Learning) को सुरक्षित रखने के लिए हमें आवश्यक बुनियादी ढाँचे का निर्माण करते हैं। इस खंड में, हम यह देखने के लिए जा रहे हैं कि कैसे इन बुनियादी उपकरणों का उपयोग करके हमारी पहली गोपनीयता को लागू करने के लिए डीप लर्निंग (Deep Learning) algorithm, फेडरेटेड लर्निंग (Federated Learning) को संरक्षित किया जाए।

लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

अनुवादक:
- Arkadip Bhattacharya - Github: [@darkmatter18](https://github.com/darkmatter18)

### फेडरेटेड लर्निंग (Federated Learning) क्या है?

यह डीप लर्निंग (Deep Learning) मॉडल को प्रशिक्षित करने का एक सरल, शक्तिशाली तरीका है। यदि आप प्रशिक्षण डेटा (Training Data) के बारे में सोचते हैं, तो यह हमेशा किसी प्रकार की संग्रह प्रक्रिया का परिणाम होता है। लोग (उपकरणों के माध्यम से) वास्तविक दुनिया में घटनाओं को रिकॉर्ड करके डेटा उत्पन्न करते हैं। 
आम तौर पर, यह डेटा एक, केंद्रीय स्थान पर एकत्रित होता है ताकि आप मशीन लर्निंग मॉडल (Machine Learniing Model) को प्रशिक्षित (train) कर सकें। Federated Learning इसे अपने सिर पर रखता है!

मॉडल में प्रशिक्षण डेटा (Training Data) लाने के बजाय (एक केंद्रीय सर्वर), आप मॉडल (Model) को प्रशिक्षण डेटा (Training Data) में लाते हैं (जहाँ भी रहते हैं)

विचार यह है कि यह जो कोई भी डेटा को केवल स्थायी प्रतिलिपि बनाने के लिए अनुमति देता है, और इस प्रकार उस पर नियंत्रण बनाए रखता है जिसके पास इसकी पहुंच है। 

बहुत अच्छा, नाह (Pretty cool, eh)?

# अनुभाग 2.1 - एक खिलौना फेडरेटेड लर्निंग (Federated Learning) उदाहरण

चलो एक खिलौना मॉडल को केंद्रीकृत तरीके से प्रशिक्षित करके शुरू करें। यह एक साधारण मॉडल के रूप में मिलता है। हमें पहले चाहिए:

- एक खिलौना डाटासेट (a toy dataset)
- एक मॉडल (a model)
- डेटा को फिट करने के लिए एक मॉडल को प्रशिक्षित करने के लिए कुछ बुनियादी प्रशिक्षण तर्क (some basic training logic for training a model to fit the data.)।

नोट: यदि यह API आपके लिए परिचित नहीं है - तो [fast.ai](http://fast.ai) पर जाएं और इस ट्यूटोरियल को जारी रखने से पहले अपना कोर्स कर लें।

In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [ ]:
train()

आखिर तुमने इसे हासिल कर ही लिया है! हमने पारंपरिक तरीके से एक बुनियादी मॉडल को प्रशिक्षित किया है। हमारे सभी डेटा को हमारी स्थानीय मशीन में एकत्रित किया जाता है और हम इसका उपयोग अपने मॉडल में अपडेट करने के लिए कर सकते हैं। फेडरेटेड लर्निंग (Federated Learning), हालांकि, इस तरह से काम नहीं करता है। तो, आइए इस उदाहरण को संशोधित करते हुए इसे Federated Learning way के रूप में करें!


तो, हमें क्या चाहिए:

- कुछ कार्यकर्ता (workers) बनाएँ
- प्रत्येक कार्यकर्ता पर प्रशिक्षण डेटा (Training Data) का Pointer प्राप्त करें
- अद्यतन प्रशिक्षण federated learning करने के लिए

    नए प्रशिक्षण कदम:
    - सही कार्यकर्ता (worker) में मॉडल भेजें
    - वहां स्थित डेटा पर ट्रेन करें
    - मॉडल वापस प्राप्त करें और अगले कार्यकर्ता (Worker) के साथ दोहराएं

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)


In [ ]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [ ]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [ ]:
train()

## बहुत बढ़िया!

तथा देखा (And voilà!)

अब हम फेडरेटेड लर्निंग (Federated Learning) का उपयोग करके एक बहुत ही सरल डीप लर्निंग मॉडल (Deep Learning Model) का प्रशिक्षण ले रहे हैं! हम प्रत्येक कार्यकर्ता को मॉडल भेजते हैं, एक नया ग्रेडिएंट उत्पन्न करते हैं, और फिर ग्रेडिएंट को हमारे स्थानीय सर्वर पर वापस लाते हैं, जहाँ हम अपने ग्लोबल मॉडल को अपडेट करते हैं। इस प्रक्रिया में कभी भी हम अंतर्निहित प्रशिक्षण डेटा तक पहुंच नहीं देखते या अनुरोध नहीं करते हैं! हम बॉब (Bob) और ऐलिस (Alice) की गोपनीयता को बचाते हैं !!!

## इस उदाहरण की कमियाँ

इसलिए, जबकि यह उदाहरण फेडरेटेड लर्निंग का एक अच्छा परिचय है, इसमें अभी भी कुछ बड़ी कमियां हैं। सबसे विशेष रूप से, जब हम `model.get()` कहते हैं और बॉब या ऐलिस से अपडेटेड मॉडल प्राप्त करते हैं, तो हम वास्तव में बॉब और ऐलिस के प्रशिक्षण डेटा के बारे में बहुत कुछ सीख सकते हैं। कुछ मामलों में, हम उनके प्रशिक्षण डेटा को पूरी तरह से बहाल कर सकते हैं!

तो, क्या करना है? वैसे, लोगों को रोजगार देने वाली पहली रणनीति केंद्रीय सर्वर (Central Server) पर अपलोड करने से पहले कई व्यक्तियों में ग्रेडिएंट का औसत है। हालाँकि, इस रणनीति को Pointer Tensor वस्तुओं के कुछ और परिष्कृत उपयोग की आवश्यकता होगी। इसलिए, अगले भाग में, हम अधिक उन्नत Pointer कार्यक्षमता के बारे में जानने के लिए कुछ समय लेने जा रहे हैं और फिर हम इस फेडरेटेड लर्निंग उदाहरण को अपग्रेड करेंगे


# बधाई हो!!! - समुदाय में शामिल होने का समय!

इस नोटबुक ट्यूटोरियल को पूरा करने पर बधाई! यदि आपने इसका आनंद लिया है और एआई और एआई आपूर्ति श्रृंखला (डेटा) के विकेन्द्रीकृत स्वामित्व के संरक्षण की ओर आंदोलन में शामिल होना चाहते हैं, तो आप निम्न तरीकों से ऐसा कर सकते हैं!

### Pysyft को Github पर Star करें!

हमारे समुदाय की मदद करने का सबसे आसान तरीका GitHub repos अभिनीत है! यह हमारे द्वारा बनाए जा रहे कूल टूल्स के बारे में जागरूकता बढ़ाने में मदद करता है।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### हमारी Slack में शामिल हों!

नवीनतम प्रगति पर अद्यतित रहने का सबसे अच्छा तरीका हमारे समुदाय में शामिल होना है! आप फॉर्म भरकर ऐसा कर सकते हैं। [http://slack.openmined.org](http://slack.openmined.org)

### एक कोड प्रोजेक्ट में शामिल हों!

हमारे समुदाय में योगदान करने का सबसे अच्छा तरीका एक कोड योगदानकर्ता बनना है! किसी भी समय आप PySyft GitHub Issues जारी करने वाले पृष्ठ पर जा सकते हैं और "प्रोजेक्ट्स" के लिए फ़िल्टर कर सकते हैं। यह आपको सभी शीर्ष स्तर के टिकट दिखाएगा कि आप किन परियोजनाओं में शामिल हो सकते हैं! यदि आप किसी परियोजना में शामिल नहीं होना चाहते हैं, लेकिन आप थोड़ी सी कोडिंग करना चाहते हैं, तो आप "अच्छा पहला अंक" चिह्नित गीथहब मुद्दों की खोज करके अधिक "वन ऑफ़" मिनी-प्रोजेक्ट्स की तलाश कर सकते हैं।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करना

यदि आपके पास हमारे कोडबेस में योगदान करने का समय नहीं है, लेकिन फिर भी समर्थन उधार देना चाहते हैं, तो आप हमारे ओपन कलेक्टिव में भी एक बैकर बन सकते हैं। सभी दान हमारी वेब होस्टिंग और अन्य सामुदायिक खर्चों जैसे कि हैकाथॉन और मीटअप की ओर जाते हैं!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)